# Module 6. Amazon SageMaker Deployment for EIA(Elastic Inference Accelerator)

---

***[주의] 본 모듈은 PyTorch EIA 1.3.1 버전에서 훈련을 수행한 모델만 배포가 가능합니다. 코드가 정상적으로 수행되지 않는다면, 프레임워크 버전을 동일 버전으로 맞춰 주시기 바랍니다.***

본 모듈에서는 Elastic Inference Accelerator(EIA)를 사용하여 모델을 배포해 보겠습니다.

### Elastic Inference Accelerator
훈련 인스턴스와 달리 실시간 추론 인스턴스는 계속 상시로 띄우는 경우가 많기에, 딥러닝 어플리케이션에서 low latency를 위해 GPU 인스턴스를 사용하면 많은 비용이 발생합니다.

Amazon Elastic Inference는 저렴하고 메모리가 작은 GPU 기반 가속기를 Amazon EC2, Amazon ECS, Amazon SageMaker에 연결할 수 있는 서비스로, Accelerator가 CPU 인스턴스에 프로비저닝되고 연결됩니다. EIA를 사용하면 GPU 인스턴스에 근접한 퍼포먼스를 보이면서 인스턴스 실행 비용을 최대 75%까지 절감할 수 있습니다. 

모든 Amazon SageMaker 인스턴스 유형, EC2 인스턴스 유형 또는 Amazon ECS 작업을 지원하며, 대부분의 딥러닝 프레임워크를 지원하고 있습니다. 지원되는 프레임워크 버전은 AWS CLI로 확인할 수 있습니다.

```bash
$ aws ecr list-images --repository-name tensorflow-inference-eia --registry-id 763104351884
$ aws ecr list-images --repository-name pytorch-inference-eia --registry-id 763104351884
$ aws ecr list-images --repository-name mxnet-inference-eia --registry-id 763104351884
```

참조: https://aws.amazon.com/ko/blogs/korea/amazon-elastic-inference-gpu-powered-deep-learning-inference-acceleration/

<br>

## 1. Inference script
---

아래 코드 셀은 `src` 디렉토리에 SageMaker 추론 스크립트인 `inference_eia.py`를 저장합니다.<br>
Module 5의 코드와 대부분 동일하지만, `model_fn()` 메서드의 구현이 다른 점을 유의해 주세요.

In [1]:
import os
import time
import sagemaker
from sagemaker.pytorch.model import PyTorchModel
role = sagemaker.get_execution_role()

In [2]:
%%writefile ./src/inference_eia.py

from __future__ import absolute_import

import argparse
import json
import logging
import os
import sys
import time
import random
from os.path import join
import numpy as np
import io
import tarfile

import boto3

from PIL import Image

import torch
import torch.distributed as dist
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import lr_scheduler
import torch.optim as optim
import torchvision
import copy
import torch.utils.data
import torch.utils.data.distributed
from torchvision import datasets, transforms, models
from torch import topk

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))

JSON_CONTENT_TYPE = 'application/json'


def model_fn(model_dir):
    logger.info("==> model_dir : {}".format(model_dir))
    traced_model = torch.jit.load(os.path.join(model_dir, 'model_eia.pth'))
    return traced_model


# Deserialize the request body
def input_fn(request_body, request_content_type='application/x-image'):
    print('An input_fn that loads a image tensor')
    print(request_content_type)
    if request_content_type == 'application/x-image':             
        img = np.array(Image.open(io.BytesIO(request_body)))
    elif request_content_type == 'application/x-npy':    
        img = np.frombuffer(request_body, dtype='uint8').reshape(137, 236)   
    else:
        raise ValueError(
            'Requested unsupported ContentType in content_type : ' + request_content_type)

    img = 255 - img
    img = img[:,:,np.newaxis]
    img = np.repeat(img, 3, axis=2)    

    test_transforms = transforms.Compose([
        transforms.ToTensor()
    ])

    img_tensor = test_transforms(img)

    return img_tensor         
        

# Predicts on the deserialized object with the model from model_fn()
def predict_fn(input_data, model):
    logger.info('Entering the predict_fn function')
    start_time = time.time()
    input_data = input_data.unsqueeze(0)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()
    input_data = input_data.to(device)
                          
    result = {}
                                                 
    with torch.no_grad():
        logits = model(input_data)
        pred_probs = F.softmax(logits, dim=1).data.squeeze()   
        outputs = topk(pred_probs, 5)                  
        result['score'] = outputs[0].detach().cpu().numpy()
        result['class'] = outputs[1].detach().cpu().numpy()
    
    print("--- Elapsed time: %s secs ---" % (time.time() - start_time))    
    return result        


# Serialize the prediction result into the response content type
def output_fn(pred_output, accept=JSON_CONTENT_TYPE):
    return json.dumps({'score': pred_output['score'].tolist(), 
                       'class': pred_output['class'].tolist()}), accept

Overwriting ./src/inference_eia.py


<br>

## 2. TorchScript Compile (Tracing)
---

PyTorch 프레임워크에서 EI를 사용하기 위해서는 [TorchScript](https://pytorch.org/docs/1.3.1/jit.html)로 모델을 컴파일해야 하며, 2020년 8월 시점에서는 PyTorch 1.3.1을 지원하고 있습니다. TorchScript는 PyTorch 코드에서 직렬화 및 최적화 가능한 모델로 컴파일하며 Python 인터프리터의 글로벌 인터프리터 잠금 (GIL)과 무관하기 때문에 Python 외의 언어에서 로드 가능하고  최적화가 용이합니다.

TorchScript로 변환하는 방법은 **tracing** 방식과 **scripting** 방식이 있으며, 본 핸즈온에서는 tracing 방식을 사용하겠습니다. <br>
참고로 tracing 방식은 샘플 입력 데이터를 모델에 입력 후 그 입력의 흐름(feedforward)을 기록하여 포착하는 메커니즘이며, scripting 방식은 모델 코드를 직접 분석해서 컴파일하는 방식입니다.

### Install dependencies

In [3]:
import sys
!{sys.executable} -m pip install --upgrade pip --trusted-host pypi.org --trusted-host files.pythonhosted.org
!{sys.executable} -m pip install https://download.pytorch.org/whl/cpu/torchvision-0.4.2%2Bcpu-cp36-cp36m-linux_x86_64.whl
!{sys.executable} -m pip install https://s3.amazonaws.com/amazonei-pytorch/torch_eia-1.3.1-cp36-cp36m-manylinux1_x86_64.whl
!{sys.executable} -m pip install graphviz==0.13.2   
!{sys.executable} -m pip install mxnet-model-server==1.0.8
!{sys.executable} -m pip install pillow==7.1.0
!{sys.executable} -m pip install sagemaker_containers
!{sys.executable} -m pip install -U sagemaker

### Compile

Tracing 방식은 특정 input을 모델에 적용했을 때 수행되면서 operation이 저장하기 때문에, 이미지 사이즈와 동일한 크기의 랜덤 입력 데이터를 모델을 적용해야 합니다.

In [4]:
import torch, os
from torchvision import models
model_dir = './model'
print("==> model_dir : {}".format(model_dir))
model = models.resnet18(pretrained=True)
last_hidden_units = model.fc.in_features
model.fc = torch.nn.Linear(last_hidden_units, 186)
model.load_state_dict(torch.load(os.path.join(model_dir, 'model.pth')))

==> model_dir : ./model


<All keys matched successfully>

In [5]:
import torch
data = torch.rand(1,3,137,236)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
input_data = data.to(device)

In [6]:
with torch.jit.optimized_execution(True, {'target_device': 'eia:0'}): 
    traced_model = torch.jit.trace(model, input_data)

/local/p4clients/pkgbuild-kCleo/workspace/build/PyTorchECL/PyTorchECL-1.x.548.0/AL2012/DEV.STD.PTHREAD/build/private/src/torch/csrc/jit/eia/eia_interface.h:52: UserWarning: Notice - No last error found
/local/p4clients/pkgbuild-kCleo/workspace/build/PyTorchECL/PyTorchECL-1.x.548.0/AL2012/DEV.STD.PTHREAD/build/private/src/torch/csrc/jit/eia/eia_interface.h:52: UserWarning: Notice - No last error found


컴파일한 모델로 로컬 환경에서 추론을 수행해 보겠습니다.

In [7]:
from src.inference_eia import model_fn, input_fn, predict_fn, output_fn
from PIL import Image
import numpy as np
import json

file_path = 'test_imgs/test_0.jpg'
with open(file_path, mode='rb') as file:
    img_byte = bytearray(file.read())
data = input_fn(img_byte)
result = predict_fn(data, traced_model)
print(result)

An input_fn that loads a image tensor
application/x-image
Entering the predict_fn function
--- Elapsed time: 0.023025035858154297 secs ---
{'score': array([0.62198836, 0.2314413 , 0.04159953, 0.02067479, 0.01897352],
      dtype=float32), 'class': array([  3,   2, 169, 168,  70])}


TorchScript 모델을 파일로 직렬화하여 저장합니다. 그런 다음, `tar.gz`로 압축하고 이 파일을 S3로 복사합니다.

In [8]:
torch.jit.save(traced_model, './model/model_eia.pth')

In [9]:
tar_filename = 'model_eia.tar.gz'
!cd model/ && tar -czvf $tar_filename model_eia.pth

model_eia.pt


In [10]:
artifacts_dir = 's3://sagemaker-us-east-1-143656149352/pytorch-training-2020-08-16-04-47-36-618/output/'
!aws s3 cp model/$tar_filename $artifacts_dir

upload: model/model_eia.tar.gz to s3://sagemaker-us-east-1-143656149352/pytorch-training-2020-08-16-04-47-36-618/output/model_eia.tar.gz


<br>

## 3. SageMaker Hosted Endpoint Inference
---

SageMaker가 관리하는 배포 클러스터를 프로비저닝하는 시간이 소요되기 때문에 추론 서비스를 시작하는 데에는 약 5~10분 정도 소요됩니다.


In [11]:
import boto3
client = boto3.client('sagemaker')
runtime_client = boto3.client('sagemaker-runtime')

In [12]:
def get_model_path(sm_client, max_results=1, name_contains='pytorch'):
    training_job = sm_client.list_training_jobs(MaxResults=max_results,
                                         NameContains=name_contains,
                                         SortBy='CreationTime', 
                                         SortOrder='Descending')
    training_job_name = training_job['TrainingJobSummaries'][0]['TrainingJobName']
    training_job_description = sm_client.describe_training_job(TrainingJobName=training_job_name)
    model_path = training_job_description['ModelArtifacts']['S3ModelArtifacts']  
    return model_path

In [18]:
#model_path = get_model_path(client, max_results=3)
model_path = os.path.join(artifacts_dir, tar_filename)
print(model_path)
endpoint_name = "endpoint-bangali-classifier-eia-{}".format(int(time.time()))

pytorch_model = PyTorchModel(model_data=model_path,
                                   role=role,
                                   entry_point='./src/inference_eia.py',
                                   framework_version='1.3.1',
                                   py_version='py3')

predictor = pytorch_model.deploy(instance_type='ml.c5.large', 
                                 initial_instance_count=1, 
                                 accelerator_type='ml.eia2.large', 
                                 endpoint_name=endpoint_name,
                                 wait=False)


s3://sagemaker-us-east-1-143656149352/pytorch-training-2020-08-16-04-47-36-618/output/model_eia.tar.gz


In [19]:
# client = boto3.client('sagemaker')
# waiter = client.get_waiter('endpoint_in_service')
# waiter.wait(EndpointName=endpoint_name)

In [20]:
import boto3
client = boto3.client('sagemaker')
runtime_client = boto3.client('sagemaker-runtime')
endpoint_name = pytorch_model.endpoint_name
client.describe_endpoint(EndpointName = endpoint_name)

{'EndpointName': 'endpoint-bangali-classifier-eia-1597846677',
 'EndpointArn': 'arn:aws:sagemaker:us-east-1:143656149352:endpoint/endpoint-bangali-classifier-eia-1597846677',
 'EndpointConfigName': 'endpoint-bangali-classifier-eia-1597846677',
 'ProductionVariants': [{'VariantName': 'AllTraffic',
   'DeployedImages': [{'SpecifiedImage': '763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-inference-eia:1.3.1-cpu-py3',
     'ResolvedImage': '763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-inference-eia@sha256:fa623bbda1358a0b50f89820f2ea3d9f1331ab5336158f510ea267088ef670be',
     'ResolutionTime': datetime.datetime(2020, 8, 19, 14, 18, 3, 917000, tzinfo=tzlocal())}],
   'CurrentWeight': 1.0,
   'DesiredWeight': 1.0,
   'CurrentInstanceCount': 1,
   'DesiredInstanceCount': 1}],
 'EndpointStatus': 'InService',
 'CreationTime': datetime.datetime(2020, 8, 19, 14, 18, 2, 136000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2020, 8, 19, 14, 25, 20, 172000, tzinfo=tzlocal())

추론을 수행합니다. (`ContentType='application/x-image'`)

In [32]:
with open(file_path, mode='rb') as file:
    img_byte = bytearray(file.read())

response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name, 
    ContentType='application/x-image',
    Accept='application/json',
    Body=img_byte
    )
print(response['Body'].read().decode())    

{"score": [0.6219883561134338, 0.23144130408763885, 0.04159948602318764, 0.02067478932440281, 0.018973516300320625], "class": [3, 2, 169, 168, 70]}


In [33]:
%timeit runtime_client.invoke_endpoint(EndpointName=endpoint_name, ContentType='application/x-image', Accept='application/json', Body=img_byte)

94.1 ms ± 6.12 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### SageMaker Hosted Endpoint Clean-up

엔드포인트를 계속 사용하지 않는다면, 불필요한 과금을 피하기 위해 엔드포인트를 삭제해야 합니다. 
SageMaker SDK에서는 `delete_endpoint()` 메소드로 간단히 삭제할 수 있으며, UI에서도 쉽게 삭제할 수 있습니다.

In [34]:
def delete_endpoint(client, endpoint_name):
    response = client.describe_endpoint_config(EndpointConfigName=endpoint_name)
    model_name = response['ProductionVariants'][0]['ModelName']

    client.delete_model(ModelName=model_name)    
    client.delete_endpoint(EndpointName=endpoint_name)
    client.delete_endpoint_config(EndpointConfigName=endpoint_name)    
    
    print(f'--- Deleted model: {model_name}')
    print(f'--- Deleted endpoint: {endpoint_name}')
    print(f'--- Deleted endpoint_config: {endpoint_name}')    

In [28]:
delete_endpoint(client, endpoint_name)